In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display
import pydub
from pydub import AudioSegment
from youtube_dl import YoutubeDL

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import json
import os
import warnings
import time
warnings.filterwarnings('ignore')
from selenium.common.exceptions import NoSuchElementException



In [107]:
def get_driver(debug=False):
    options = Options()

    if not debug:
        options.add_argument("--headless")
    else:
        options.add_argument("--window-size=1920,1080")

    options.add_argument("--disable-notifications")
    options.add_argument('--disable-gpu')
    options.add_argument("--lang=en-US")
    driv_path = '../webdriver/chromedriver.exe'
    input_driver = webdriver.Chrome(options=options, executable_path=driv_path)
    return input_driver

In [278]:
def get_fresh_element(driver, class_name, i,vid=False, xpath = '//tbody'):
    play_list = driver.find_element_by_xpath(xpath)
    try:
        if vid:
            return play_list.find_elements_by_class_name(class_name)[i].find_element_by_css_selector('a').get_attribute('href')
        return play_list.find_elements_by_class_name(class_name)[i].text
    except NoSuchElementException:
        pass

In [279]:
def get_playlist(driver,song_list, genre):
    play_list = driver.find_element_by_xpath('//tbody')
    yt_vids = play_list.find_elements_by_class_name('chartlist-play')
    
    for i in range(len(yt_vids)):
        song_dict = {}
        song_dict['genre'] = genre
        song_dict['video_link'] = get_fresh_element(driver,'chartlist-play',i, vid=True )
        song_dict['artist']  = get_fresh_element(driver,'chartlist-artist',i )       
        song_dict['song']  = get_fresh_element(driver,'chartlist-name',i )
        song_list.append(song_dict)
    return song_list

In [350]:
def scrape_last_fm(class_name, url = 'https://www.last.fm/music'):
    merged_song_list = []
    driver = get_driver(debug=True)
    driver.get(url)
    time.sleep(2)
    genre_tag = driver.find_element_by_class_name('music-section')
    genres = genre_tag.find_elements_by_class_name(class_name)
    if genres[0].text =='':
        genres = genres[1:]
    for x in range(len(genres)):
        genre_tag = driver.find_element_by_class_name('music-section')
        genres = genre_tag.find_elements_by_class_name(class_name)
        if genres[0].text =='':
            genres = genres[1:]
        genre = genres[x].text
        
        driver.execute_script("arguments[0].click();", genres[x])
        time.sleep(2)
        more_tracks = driver.find_elements_by_xpath('//p[@class="more-link"]')
        driver.execute_script("window.scrollTo(0,1000)")
        driver.execute_script("arguments[0].click();", more_tracks[1].find_element_by_css_selector('a'))
        time.sleep(2)
        song_list = get_playlist(driver, [], genre)
        time.sleep(2)
        nxt_page = driver.find_element_by_class_name('pagination-next')
        driver.execute_script("arguments[0].click();", nxt_page.find_element_by_css_selector('a'))
        time.sleep(2)
        song_list_100 = get_playlist(driver, song_list, genre)
        driver.back()
        time.sleep(1)
        driver.back()
        time.sleep(1)
        driver.back()
        time.sleep(2)
        merged_song_list.extend(song_list_100)
        print('Finished {} Genre'.format(genre))
    return merged_song_list

In [351]:
sl_featured = scrape_last_fm('music-featured-item-heading-link')
song_list_others = scrape_last_fm('music-more-tags-tag-link')

Finished Electronic Genre
Finished Metal Genre
Finished rock Genre
Finished hip-hop Genre
Finished indie Genre
Finished jazz Genre
Finished reggae Genre
Finished british Genre
Finished punk Genre
Finished 80s Genre
Finished dance Genre
Finished acoustic Genre
Finished rnb Genre
Finished hardcore Genre
Finished country Genre
Finished blues Genre
Finished alternative Genre
Finished classical Genre
Finished rap Genre
Finished country Genre


In [359]:
df_combined = pd.concat([pd.DataFrame(sl_featured),pd.DataFrame(song_list_others)], ignore_index = True)
df_combined.to_csv('../data/complete_playlist.csv', index= False)

In [375]:
df_combined_prep = df_combined.drop_duplicates(subset = ['genre', 'artist', 'song'])
df_combined_prep = df_combined_prep[~df_combined.genre.isin(['80s', 'british'])]
df_combined_prep = df_combined_prep.dropna(subset = ['video_link'])
df_combined_prep.to_csv('../data/filtered_playlist.csv', index= False)

In [366]:
len(df_combined),len(df_combined_prep)

(2000, 1631)

In [374]:
df_combined_prep.groupby('genre').size()

genre
Electronic      99
Metal           99
acoustic        96
alternative    100
blues           98
classical       89
country         99
dance           98
hardcore        67
hip-hop         95
indie           99
jazz            97
punk           100
rap             97
reggae          98
rnb            100
rock           100
dtype: int64

In [2]:
df_combined_prep = pd.read_csv('../data/filtered_playlist.csv')

In [3]:
def download_clip(url, fname):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': fname,
        'noplaylist': True,
        'continue_dl': True,
        'quiet': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '320', 
        }]
    }
    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.cache.remove()
            info_dict = ydl.extract_info(url, download=False)
            ydl.prepare_filename(info_dict)
            ydl.download([url])
            return fname
    except Exception:
        return 'Error' 

In [5]:
download_path = []
for i,row in df_combined_prep.iterrows():
    dpath = '../data/wav_files/{}/'.format(row['genre'])
    os.makedirs(dpath, exist_ok=True)
    fname= '{0}{1}_{2}.wav'.format(dpath,row['artist'], row['song'])
    download_path.append(download_clip(row['video_link'], fname))

Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] fe4EK4HSPkI: Downloading webpage
[youtube] Downloading just video fe4EK4HSPkI because of --no-playlist
[youtube] fe4EK4HSPkI: Downloading player fae06c11
[youtube] fe4EK4HSPkI: Downloading webpage
[youtube] Downloading just video fe4EK4HSPkI because of --no-playlist
[download] Destination: ..\data\wav_files\Electronic\MGMT_Kids.wav
[download] 100% of 5.67MiB in 01:52                 
[ffmpeg] Post-process file ..\data\wav_files\Electronic\MGMT_Kids.wav exists, skipping
Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] u7K72X4eo_s: Downloading webpage
[youtube] Downloading just video u7K72X4eo_s because of --no-playlist
[youtube] u7K72X4eo_s: Downloading player fae06c11
[youtube] u7K72X4eo_s: Downloading webpage
[youtube] Downloading just video u7K72X4eo_s because of --no-playlist
[download] Destination: ..\data\wav_files\Electronic\Massive Attack_Teardrop.wav
[download] 100% of 4.55MiB in 00:00           

ERROR: unable to download video data: HTTP Error 403: Forbidden


Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] dwDns8x3Jb4: Downloading webpage
[youtube] Downloading just video dwDns8x3Jb4 because of --no-playlist
[youtube] dwDns8x3Jb4: Downloading player fae06c11
[youtube] dwDns8x3Jb4: Downloading webpage
[youtube] Downloading just video dwDns8x3Jb4 because of --no-playlist
[download] Destination: ..\data\wav_files\Electronic\Daft Punk_Around the World.wav
[download] 100% of 8.09MiB in 02:44                 
[ffmpeg] Post-process file ..\data\wav_files\Electronic\Daft Punk_Around the World.wav exists, skipping
Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] hbe3CQamF8k: Downloading webpage
[youtube] Downloading just video hbe3CQamF8k because of --no-playlist
[youtube] hbe3CQamF8k: Downloading player fae06c11
[youtube] hbe3CQamF8k: Downloading webpage
[youtube] Downloading just video hbe3CQamF8k because of --no-playlist
[download] Destination: ..\data\wav_files\Electronic\Massive Attack_Angel.wav
[download] 100%

[ffmpeg] Post-process file ..\data\wav_files\Electronic\The Chemical Brothers_Galvanize.wav exists, skipping
Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] SC6-TiN19uE: Downloading webpage
[youtube] Downloading just video SC6-TiN19uE because of --no-playlist
[youtube] SC6-TiN19uE: Downloading webpage
[youtube] Downloading just video SC6-TiN19uE because of --no-playlist
[download] Destination: ..\data\wav_files\Electronic\Moby_Natural Blues.wav
[download] 100% of 3.96MiB in 00:54                  
[ffmpeg] Correcting container in "..\data\wav_files\Electronic\Moby_Natural Blues.wav"
[ffmpeg] Post-process file ..\data\wav_files\Electronic\Moby_Natural Blues.wav exists, skipping
Removing cache dir C:\Users\swami/.cache\youtube-dl ..
[youtube] kxWFyvTg6mc: Downloading webpage
[youtube] Downloading just video kxWFyvTg6mc because of --no-playlist
[youtube] kxWFyvTg6mc: Downloading player fae06c11
[youtube] kxWFyvTg6mc: Downloading webpage
[youtube] Downloading just video k

[download] Destination: ..\data\wav_files\Electronic\The Knife_Heartbeats.wav
[download] 100% of 3.53MiB in 01:18                 
[ffmpeg] Correcting container in "..\data\wav_files\Electronic\The Knife_Heartbeats.wav"
[ffmpeg] Post-process file ..\data\wav_files\Electronic\The Knife_Heartbeats.wav exists, skipping
Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] UclCCFNG9q4: Downloading webpage
[youtube] Downloading just video UclCCFNG9q4 because of --no-playlist
[youtube] UclCCFNG9q4: Downloading player fae06c11
[youtube] UclCCFNG9q4: Downloading webpage
[youtube] Downloading just video UclCCFNG9q4 because of --no-playlist
[download] Destination: ..\data\wav_files\Electronic\Gorillaz_Clint Eastwood.wav
[download] 100% of 4.23MiB in 01:01                  
[ffmpeg] Correcting container in "..\data\wav_files\Electronic\Gorillaz_Clint Eastwood.wav"
[ffmpeg] Post-process file ..\data\wav_files\Electronic\Gorillaz_Clint Eastwood.wav exists, skipping
Removing cache dir C:

...

ERROR: unable to download video data: HTTP Error 403: Forbidden


Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] e5PnuIRnJW8: Downloading webpage
[youtube] Downloading just video e5PnuIRnJW8 because of --no-playlist
[youtube] e5PnuIRnJW8: Downloading player fae06c11
[youtube] e5PnuIRnJW8: Downloading webpage
[youtube] Downloading just video e5PnuIRnJW8 because of --no-playlist
[download] Destination: ..\data\wav_files\rap\Nas_The World Is Yours.wav
[download] 100% of 3.83MiB in 01:20                 
[ffmpeg] Correcting container in "..\data\wav_files\rap\Nas_The World Is Yours.wav"
[ffmpeg] Post-process file ..\data\wav_files\rap\Nas_The World Is Yours.wav exists, skipping
Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] RtSDWq6HsJE: Downloading webpage
[youtube] Downloading just video RtSDWq6HsJE because of --no-playlist
[youtube] RtSDWq6HsJE: Downloading player fae06c11
[youtube] RtSDWq6HsJE: Downloading webpage
[youtube] Downloading just video RtSDWq6HsJE because of --no-playlist
[download] Destination: ..\data

ERROR: unable to download video data: HTTP Error 403: Forbidden


Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] WeYsTmIzjkw: Downloading webpage
[youtube] Downloading just video WeYsTmIzjkw because of --no-playlist
[youtube] WeYsTmIzjkw: Downloading player fae06c11
[youtube] WeYsTmIzjkw: Downloading webpage
[youtube] Downloading just video WeYsTmIzjkw because of --no-playlist
[download] Destination: ..\data\wav_files\rap\Afroman_Because I Got High.wav
[download] 100% of 3.05MiB in 01:02                 
[ffmpeg] Correcting container in "..\data\wav_files\rap\Afroman_Because I Got High.wav"
[ffmpeg] Post-process file ..\data\wav_files\rap\Afroman_Because I Got High.wav exists, skipping
Removing cache dir C:\Users\swami/.cache\youtube-dl ...
[youtube] U2waT9TxPU0: Downloading webpage
[youtube] Downloading just video U2waT9TxPU0 because of --no-playlist
[youtube] U2waT9TxPU0: Downloading player fae06c11
[youtube] U2waT9TxPU0: Downloading webpage
[youtube] Downloading just video U2waT9TxPU0 because of --no-playlist
[download] Destinat

ERROR: unable to download video data: HTTP Error 403: Forbidden
